## Obtain the last trained model and predict scores for 10k random variants

In [88]:
from sklearn.externals import joblib
from kipoi_cadd.data import CaddDataset
from kipoi_cadd.utils import load_pickle
from scipy.sparse import csr_matrix
import cyvcf2
import numpy as np

In [93]:
training_dir = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/"
kipoi_features_dir = "/s/project/kipoi-cadd/data/processed/v1.3/kipoi_features/"
lmdb_dir = training_dir + "lmdb"
model_file = "/tmp/gin-train/62cdc2663f484ba9a9614dcd8283aad9/model.h5"
shuff_10k_file = training_dir + "shuffle_splits/shuffled_index_10k.pkl"
vars_to_compare = kipoi_features_dir + "vars_to_compare.vcf"

In [94]:
def variants_to_basic_vcf(variants, output_file=None):
    vcf_string = ""
    for var in variants:
        if isinstance(var, cyvcf2.Variant):
            vcf_string += var.CHROM + "\t" + var.POS + "\t" + var.ID + "\t" + var.REF + "\t" + var.ALT[0] + "\n"
        else:
            chrom, pos, ref, alt = var.split(":")
            vcf_string += chrom + "\t" + pos + "\t.\t" + ref + "\t" + alt.split("'")[1] + "\n"
    
    if output_file is not None:
        with open(output_file, "w") as f:
            f.write(vcf_string)
        return None
    else:
        return vcf_string

variants_to_basic_vcf(shuff_10k, vars_to_compare)

In [10]:
shuff_10k = load_pickle(shuff_10k_file)
shuff_10k.head()

3299    1:1088349:C:['T']
3730    1:1106143:T:['C']
9371    1:1967148:T:['C']
8435    1:1840772:T:['G']
1694     1:991451:C:['T']
dtype: object

In [3]:
clf = joblib.load(model_file)

In [13]:
ds = CaddDataset(lmdb_dir, shuff_10k_file)

In [15]:
X, y = ds.load_all(shuffle=False)

100%|██████████| 157/157 [00:00<00:00, 526.71it/s]


In [20]:
# TODO: Why is this y.dtype = dtype('int64') ???
y.dtype

dtype('int64')

In [23]:
# Do the same process as the trainer
X_sparse = csr_matrix(X, shape=None, dtype=np.float32, copy=True)
X = X_sparse
y = y.astype(np.float32)
del X_sparse

In [25]:
y.dtype

dtype('float32')

In [30]:
y_pred = clf.predict_proba(X)
y_pred

array([[0.62682846, 0.37317154],
       [0.46786106, 0.53213894],
       [0.53893632, 0.46106368],
       ...,
       [0.58041913, 0.41958087],
       [0.48847238, 0.51152762],
       [0.66130594, 0.33869406]])

## Try read gzipped vcf file
Used this command:
`vep --input_file myVariants.vcf.gz --output_file myOutput.vcf --vcf --fields "CADD_PHRED,CADD_RAW" --cache --dir_cache /opt/modules/i12g/ensembl-vep/92/cachedir --assembly GRCh37 --offline --force_overwrite --plugin CADD,{path-to}/whole_genome_SNVs_1.3.tsv.gz,{path-to}/InDels_1.3.tsv.gz`

The INFO field presents itself as follows: `<key>=<data>[,data]`.

In [96]:
vcf_gz = "/s/databases/cadd/CADD_v1.3/test_output_1.3.vcf.gz"
test_vcf_gz = "/s/databases/cadd/CADD_v1.3/test.vcf"
my_vcf_output = kipoi_features_dir + "vars_to_compare_output_1.3.vcf"

In [32]:
from cyvcf2 import VCF

In [79]:
myVar = None
for var in VCF(vcf_gz):
    myVar = var
    scores = [(float(x.split("|")[0]), float(x.split("|")[1])) for x in myVar.INFO.get('CSQ').split(",")]
    cadd_phred, cadd_raw = scores[0][0], scores[0][1]
    print(cadd_phred, cadd_raw)

4.364 0.170357
7.346 0.488985
7.333 0.487299
2.104 -0.053137
1.315 -0.161708
6.129 0.345946
3.435 0.083762
4.25 0.159686
8.461 0.644741
27.2 5.824971
27.3 5.856453
1.045 -0.210303
22.3 3.030159
5.45 0.275144
18.7 2.381732
23.7 4.078581
25.9 5.342419
34.0 7.472678
6.93 0.437075


In [69]:
header_info = "Allele|Consequence|IMPACT|SYMBOL|Gene|Feature_type|Feature|BIOTYPE|EXON|INTRON|HGVSc|HGVSp|cDNA_position|CDS_position|Protein_position|Amino_acids|Codons|Existing_variation|DISTANCE|STRAND|FLAGS|SYMBOL_SOURCE|HGNC_ID|CADD_PHRED|CADD_RAW"
len(header_info.split("|"))

25

In [84]:
! head -10 {test_vcf_gz}

1	14673	rs11582131	G	C
1	14677	.	G	A
1	14699	rs11490464	C	G
1	14907	rs6682375	A	G
1	14930	rs6682385	A	G
1	15906	.	A	G
1	15922	rs80168857	A	G
1	15956	.	G	A
1	16949	rs708637	A	C
1	3519049	.	AC	A


In [98]:
! head -100 {my_vcf_output}

##fileformat=VCFv4.1
##VEP="v92" time="2019-01-02 04:28:27" cache="/opt/modules/i12g/ensembl-vep/92/cachedir/homo_sapiens/92_GRCh37" 1000genomes="phase3" COSMIC="81" ClinVar="201706" ESP="20141103" HGMD-PUBLIC="20164" assembly="GRCh37.p13" dbSNP="150" gencode="GENCODE 19" genebuild="2011-04" gnomAD="170228" polyphen="2.2.2" regbuild="1.0" sift="sift5.2.2"
##INFO=<ID=CSQ,Number=.,Type=String,Description="Consequence annotations from Ensembl VEP. Format: CADD_PHRED|CADD_RAW">
##CADD_PHRED=PHRED-like scaled CADD score
##CADD_RAW=Raw CADD score
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO
1	1088349	.	C	T	.	.	CSQ=|
1	1106143	.	T	C	.	.	CSQ=|,|,|,|,|,|,|,|
1	1967148	.	T	C	.	.	CSQ=|
1	1840772	.	T	G	.	.	CSQ=|
1	991451	.	C	T	.	.	CSQ=|,|,|,|,|,|
1	1308686	.	C	T	.	.	CSQ=|,|,|,|,|,|
1	1747349	.	C	T	.	.	CSQ=|,|,|,|,|
1	1317646	.	C	T	.	.	CSQ=|,|,|,|,|,|,|,|,|,|,|,|
1	2007148	.	C	T	.	.	CSQ=|,|,|,|,|,|,|,|,|,|
1	1986400	.	T	A	.	.	CSQ=|,|,|,|,|,|,|
1	1253092	.	G	C	.	.	CSQ=|,|,|,|,|,|,|,|,|,|,|,|,|,|,|,|,|,|,|